# Porto Seguro's Safe Driving Prediction

Porto Seguro, one of Brazil’s largest auto and homeowner insurance companies, completely agrees. Inaccuracies in car insurance company’s claim predictions raise the cost of insurance for good drivers and reduce the price for bad ones.

In the [Porto Seguro Safe Driver Prediction competition](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction), the challenge is to build a model that predicts the probability that a driver will initiate an auto insurance claim in the next year. While Porto Seguro has used machine learning for the past 20 years, they’re looking to Kaggle’s machine learning community to explore new, more powerful methods. A more accurate prediction will allow them to further tailor their prices, and hopefully make auto insurance coverage more accessible to more drivers.

Lucky for you, a machine learning model was built to solve the Porto Seguro problem by the data scientist on your team. The solution notebook has steps to load data, split the data into test and train sets, train, evaluate and save a LightGBM model that will be used for the future challenges.

#### Hint: use shift + enter to run the code cells below. Once the cell turns from [*] to [#], you can be sure the cell has run. 

## Import Needed Packages

Import the packages needed for this solution notebook. The most widely used packages for machine learning for [scikit-learn](https://scikit-learn.org/stable/), [pandas](https://pandas.pydata.org/docs/getting_started/index.html#getting-started), and [numpy](https://numpy.org/). These packages have various features, as well as a lot of clustering, regression and classification algorithms that make it a good choice for data mining and data analysis. In this notebook, we're using a training function from [lightgbm](https://lightgbm.readthedocs.io/en/latest/index.html).

In [1]:
import os
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
import joblib
from sklearn import metrics

## Load Data

Load the training dataset from the ./data/ directory. Df.shape() allows you to view the dimensions of the dataset you are passing in. If you want to view the first 5 rows of data, df.head() allows for this.

In [2]:
DATA_DIR = "./data"
data_df = pd.read_csv(os.path.join(DATA_DIR, 'porto_seguro_safe_driver_prediction_input.csv'))
print(data_df.shape)
data_df.head()

(189011, 59)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9.0,1.0,5.0,8.0,0.0,1.0,1.0,0.0,0.0,1.0
1,9,0,1,1,7,0,0,0,0,1,...,3.0,1.0,1.0,9.0,0.0,1.0,1.0,0.0,1.0,0.0
2,13,0,5,4,9,1,0,0,0,1,...,4.0,2.0,7.0,7.0,0.0,1.0,1.0,0.0,1.0,0.0
3,16,0,0,1,2,0,0,1,0,0,...,2.0,2.0,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17,0,0,2,0,1,0,1,0,0,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0


## Split Data into Train and Validatation Sets

Partitioning data into training, validation, and holdout sets allows you to develop highly accurate models that are relevant to data that you collect in the future, not just the data the model was trained on. 

In machine learning, features are the measurable property of the object you’re trying to analyze. Typically, features are the columns of the data that you are training your model with minus the label. In machine learning, a label (categorical) or target (regression) is the output you get from your model after training it.

In [3]:
features = data_df.drop(['target', 'id'], axis = 1)
labels = np.array(data_df['target'])
features_train, features_valid, labels_train, labels_valid = train_test_split(features, labels, test_size=0.2, random_state=0)

train_data = lightgbm.Dataset(features_train, label=labels_train)
valid_data = lightgbm.Dataset(features_valid, label=labels_valid, free_raw_data=False)

## Train Model

A machine learning model is an algorithm which learns features from the given data to produce labels which may be continuous or categorical ( regression and classification respectively ). In other words, it tries to relate the given data with its labels, just as the human brain does.

In this cell, the data scientist used an algorithm called [LightGBM](https://lightgbm.readthedocs.io/en/latest/), which primarily used for unbalanced datasets. AUC will be explained in the next cell.

In [4]:
parameters = {
    'learning_rate': 0.02,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': 4
}
    
model = lightgbm.train(parameters,
                           train_data,
                           valid_sets=valid_data,
                           num_boost_round=500,
                           early_stopping_rounds=20)

[1]	valid_0's auc: 0.578288
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.598426
[3]	valid_0's auc: 0.601087
[4]	valid_0's auc: 0.60043
[5]	valid_0's auc: 0.600569
[6]	valid_0's auc: 0.6036
[7]	valid_0's auc: 0.605403
[8]	valid_0's auc: 0.604875
[9]	valid_0's auc: 0.607566
[10]	valid_0's auc: 0.610681
[11]	valid_0's auc: 0.61153
[12]	valid_0's auc: 0.61371
[13]	valid_0's auc: 0.614857
[14]	valid_0's auc: 0.615266
[15]	valid_0's auc: 0.615536
[16]	valid_0's auc: 0.614975
[17]	valid_0's auc: 0.61494
[18]	valid_0's auc: 0.615227
[19]	valid_0's auc: 0.615527
[20]	valid_0's auc: 0.61621
[21]	valid_0's auc: 0.616424
[22]	valid_0's auc: 0.617489
[23]	valid_0's auc: 0.617473
[24]	valid_0's auc: 0.617351
[25]	valid_0's auc: 0.617745
[26]	valid_0's auc: 0.618574
[27]	valid_0's auc: 0.61947
[28]	valid_0's auc: 0.61907
[29]	valid_0's auc: 0.619527
[30]	valid_0's auc: 0.619871
[31]	valid_0's auc: 0.619596
[32]	valid_0's auc: 0.618677
[33]	valid_0's auc: 0.619048


## Evaluate Model

Evaluating performance is an essential task in machine learning. In this case, because this is a classification problem, the data scientist elected to use an AUC - ROC Curve. When we need to check or visualize the performance of the multi - class classification problem, we use AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curve. It is one of the most important evaluation metrics for checking any classification model’s performance.

<img src="https://www.researchgate.net/profile/Oxana_Trifonova/publication/276079439/figure/fig2/AS:614187332034565@1523445079168/An-example-of-ROC-curves-with-good-AUC-09-and-satisfactory-AUC-065-parameters.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 12px; width: 320px; height: 239px;" />

In [5]:
predictions = model.predict(valid_data.data)
fpr, tpr, thresholds = metrics.roc_curve(valid_data.label, predictions)
model_metrics = {"auc": (metrics.auc(fpr, tpr))}
print(model_metrics)

{'auc': 0.6235463621043786}


## Save Model
 
In machine learning, we need to save the trained models in a file and restore them in order to reuse it to compare the model with other models, to test the model on a new data. The saving of data is called Serializaion, while restoring the data is called Deserialization.

In [6]:
model_name = "porto_seguro_safe_driver_model.pkl"
joblib.dump(value=model, filename=model_name)

['porto_seguro_safe_driver_model.pkl']